# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
# @title Install (and import) the dependencies
%%capture
!pip install dlt[duckdb]
import dlt
import duckdb

In [ ]:
## 1) Use a generator

# Question 1
result = 0
limit = 5
generator = square_root_generator(limit)
for sqrt_value in generator:
    result += sqrt_value

# Print answer of Question 1
print(f'Q1 : {result}')


# Question 2
limit = 13
generator = square_root_generator(limit)
for _ in range(limit):
    result = next(generator)

# Print answer of Question 2
print(f'Q2 : {result}')

Q1 : 8.382332347441762
Q2 : 3.605551275463989


In [ ]:
## 2) Append a generator to a table with existing data

# Load the first generator
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_dataset')
info = pipeline.run(people_1(), table_name="people_append", write_disposition="replace")

# Load the second generator (append)
info = pipeline.run(people_2(), table_name="people_append", write_disposition="append")

# Show the outcome
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Sum of all ages of people
print('Sum of all ages of people :')
display(conn.sql("SELECT SUM(age) AS age_sum FROM people_append").df())

# Display "people_append" table
print('\n\n"people_append" table:')
display(conn.sql("SELECT * FROM people_append").df())

Sum of all ages of people :


,age_sum
0,353.0




"people_append" table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707979769.769466,7r9+evRmfaCc8g,None
1,2,Person_2,27,City_A,1707979769.769466,F06UFgnS2lzh+Q,None
2,3,Person_3,28,City_A,1707979769.769466,9bAOum9pKT4C+w,None
3,4,Person_4,29,City_A,1707979769.769466,E487vh2Us7zPcA,None
4,5,Person_5,30,City_A,1707979769.769466,/faGUz9PX5P83A,None
5,3,Person_3,33,City_B,1707979770.3712094,NRw1LZp65Sfs+g,Job_3
6,4,Person_4,34,City_B,1707979770.3712094,zwn6WphO4GTY1Q,Job_4
7,5,Person_5,35,City_B,1707979770.3712094,S8hezO7WY41fgA,Job_5
8,6,Person_6,36,City_B,1707979770.3712094,Y1/UhpuVnjMexQ,Job_6
9,7,Person_7,37,City_B,1707979770.3712094,Ql/HMvrlnPtfqw,Job_7


In [ ]:
## 3) Merge a generator

# Load the first generator
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_dataset')
info = pipeline.run(people_1(), table_name="people_merge", write_disposition="replace", primary_key='ID')

# Load the second generator (merge)
info = pipeline.run(people_2(), table_name="people_merge", write_disposition="merge", primary_key='ID')

# Show the outcome
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Sum of all ages of people
print('Sum of all ages of people :')
display(conn.sql("SELECT SUM(age) AS age_sum FROM people_merge").df())

# Display "people_merge" table
print('\n\n"people_merge" table:')
display(conn.sql("SELECT * FROM people_merge").df())

Sum of all ages of people :


,age_sum
0,266.0




"people_merge" table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707979771.1796038,2B1Ig+CMVTkD7w,None
1,2,Person_2,27,City_A,1707979771.1796038,ys+wTxxLQJFFXA,None
2,5,Person_5,35,City_B,1707979771.8324704,1YpsYKgTR7V6Zw,Job_5
3,7,Person_7,37,City_B,1707979771.8324704,OZY3hyTuOhcDTw,Job_7
4,4,Person_4,34,City_B,1707979771.8324704,8VcDZcUgCoSb9w,Job_4
5,8,Person_8,38,City_B,1707979771.8324704,wsoiTDXPvGi9WA,Job_8
6,3,Person_3,33,City_B,1707979771.8324704,ErLIqe6pa6CxdQ,Job_3
7,6,Person_6,36,City_B,1707979771.8324704,aZaTzYroM4tsJA,Job_6


In [ ]:
# Display all loaded tables in duckdb
print('Loaded tables:')
display(conn.sql("show tables").df())

Loaded tables:


,name
0,_dlt_loads
1,_dlt_pipeline_state
2,_dlt_version
3,people_append
4,people_merge


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX